In [1]:
import os
import re
import pymupdf
import django_setup
from IPython.display import Markdown, display
from bs4 import BeautifulSoup as bs

from django.conf import settings

gemini_api_key = settings.GEMINI_API_KEY

In [2]:
import google.generativeai as genai
genai.configure(api_key=gemini_api_key)


In [3]:
def get_model_configured():
    generation_config = {
      "temperature": 0.1,
      "top_p": 0.95,
      "top_k": 40,
      #"max_output_tokens": 8192,
      "response_mime_type": "application/json",
    }

    model = genai.GenerativeModel(
      model_name="gemini-2.0-flash-exp",
      generation_config=generation_config,
    )

    return model

model = get_model_configured()


In [4]:
def clean_text(text, _normalizes=None):
    txt = text
    try:
        normalizes = _normalizes or (
            ('\n ?\d+ ?/ ?\d+ ?\n', '\n'),
            ('[ ]{2,}', ' '),
            (' \\n', '\n'),
            #('([^\.]|\S)\n(\S)', r'\1 \2'),
            ('(\w)\n(.)', r'\1 \2'),
            ('(,)\n(.)', r'\1 \2'),
            #('()\n()', r'\1 \2'),
            ('\n\n', '\n'),
            ('-\n', '-'),
            ('^\n', ''),
            ('–', '-'),
            ('•', '*'),
            ('[“”]', '"'),
            #('', ''),
            #('', ''),
            #('', ''),
            #('', ''),
            #('', ''),
        )

        for regex, new in normalizes:
            search = re.search(regex, text)
            while search:
                text = re.sub(regex, new, text)
                search = re.search(regex, text)
        return text
    except:
        return txt


In [9]:
def make_prompt(text):
    prompt = ("""    Você é um especialista em Gestão Pública Municipal com mais de 10 anos de conhecimento teórico e atividade prática.
        Analise o conteúdo de <CONTEXTO></CONTEXTO> e escreva um JSON com as seguintes chaves: "temas", "autores", "destinatarios", "possiveis_destinatarios", "localidades", "analise" e "resumo".
        Para preencher as chaves, siga as instruções abaixo em que, respectivamente, se referem às chaves acima citadas:
        1) O  texto em análise aborda a importância e faz solicitações à seus destinatários para a sociedade local. Em <TEMAS></TEMAS> consta uma lista com diversos temas da gestão pública municipal, classifique o texto com os itens de <TEMAS></TEMAS>, utilize para a classificação quantos itens forem necessários e coloque sua seleção em formato de lista na chave "temas".
        2) O texto em análise apresenta um ou mais autores que defendem a importância de uma ou mais ações para a sociedade. Identifique esses autores de forma objetiva, ou seja, nome e cargo dentro de um dicionário para cada autor. Coloque sua resposta à esta pergunta na chave "autores" no formato de lista, mesmo que seja identificado apenas um autor.
        3) Se o texto em análise apresentar um ou mais destinatários das ações defendidas pelos autores. Identifique esses destinatários com seu nome, cargo e instituição a qual pertence. Explique a importância das ações defendidas pelos autores para esses destinatários no contexto de seus cargos e instituições. Utilize em sua resposta as chaves: "nome", "cargo", "instituicao", "importancia". Coloque sua resposta à esta pergunta na chave "destinatarios" no formato de lista, mesmo que seja identificado apenas um destinatário.
        4) Com base nas diversas áreas da gestão pública municipal, formule uma lista de possíveis instituições destinatárias não citadas pelos autores e justifique. Use as chaves "instituicao" e "justificativa". Coloque sua resposta à esta pergunta na chave "possiveis_destinatarios" no formato de lista.
        5) Caso o texto em análise apresente localidades, identifique essas localidades, sejam elas logradouros públicos, bairros ou mesmo prédios públicos e/ou privados, utilizando para isso a chave "nome", e adicionando o tipo identificado, utilizando a chave "tipo". Coloque sua resposta à esta pergunta na chave "localidades" no formato de lista.
        6) Formule a defesa do texto analisado com no máximo 150 palavras sobre a importância das ações defendidas pelos autores para os destinatários identificados. É muito importante que escreva essa defesa com um texto leve de forma que uma pessoa leiga no assunto entenda. Coloque sua resposta à esta pergunta na chave "analise" no formato de markdown.
        7) Resuma o texto em uma única frase dizendo qual o benefício está sendo pedido para que localidade está sendo pedido sem citar autores e destinatários. Coloque sua resposta à esta pergunta na chave "resumo".
        <CONTEXTO>{context}</CONTEXTO>
        <TEMAS><ITEM>Assistência Social e Inclusão</ITEM><ITEM>Atendimento ao Cidadão</ITEM><ITEM>Bem-Estar Animal</ITEM><ITEM>Conselhos Municipais</ITEM><ITEM>Convênios e Parcerias</ITEM><ITEM>Cultura e Patrimônio</ITEM><ITEM>Desenvolvimento Econômico e Social</ITEM><ITEM>Economia Solidária</ITEM><ITEM>Educação</ITEM><ITEM>Espaços Públicos</ITEM><ITEM>Esporte e Lazer</ITEM><ITEM>Finanças e Orçamento</ITEM><ITEM>Gerenciamento de Riscos</ITEM><ITEM>Gestão de Contratos e Licitações</ITEM><ITEM>Gestão de Pessoas e Servidores</ITEM><ITEM>Governança e Gestão Pública</ITEM><ITEM>Infraestrutura Urbana e Mobilidade</ITEM><ITEM>Legislação e Normas</ITEM><ITEM>Meio Ambiente e Sustentabilidade</ITEM><ITEM>Participação e Controle Social</ITEM><ITEM>Qualidade de Vida</ITEM><ITEM>Regularização Fundiária</ITEM><ITEM>Relações Institucionais</ITEM><ITEM>Saneamento Básico</ITEM><ITEM>Saúde Pública</ITEM><ITEM>Segurança Pública e Defesa Civil</ITEM><ITEM>Tecnologia da Informação</ITEM></TEMAS>
    """).format(context=text)

    while '  ' in prompt:
        prompt = prompt.replace('  ', ' ')

    return prompt


In [18]:
from sapl.materia.models import MateriaLegislativa

materias = MateriaLegislativa.objects.filter(numero=36, ano=2025, tipo_id=3)

for materia in materias[:1]:
    doc = pymupdf.open(materia.texto_original.path)
    text = ' '.join([page.get_text() for page in doc])
    text = clean_text(text)

prompt = make_prompt(text)

answer = model.generate_content(prompt)

In [19]:
import json

try:
    data = {}
    data['id'] = materia.id
    data['epigrafe'] = str(materia)
    data['ementa'] = materia.ementa
    data.update(json.loads(answer.text))
    data_json = json.dumps(data, indent=4, ensure_ascii=False)
    open(f'data/002_output_{materia.numero}.json', 'w').write(data_json)
except:
    print('Erro ao converter json')
    print(answer.text)
    open('data/002_error.txt', 'w').write(answer.text)

In [23]:
import yaml

yaml_string = """PROMPTs:
  - id: 2
    status: True
    temperature: 0.1
    top_p: 0.95
    top_k: 40
    text: |+
      Você é um especialista em Gestão Pública Municipal com mais de 10 anos de conhecimento teórico e atividade prática.
      Analise o conteúdo de <CONTEXTO></CONTEXTO> e escreva um JSON com as seguintes chaves: "temas", "autores", "destinatarios", "possiveis_destinatarios", "localidades" e "analise".
      Para preencher as chaves, siga as instruções abaixo em que, respectivamente, se referem às chaves acima citadas:
      1) Dentro das diversas áreas da gestão pública municipal, o texto em análise aborda a importância e faz solicitações à seus destinatários para a sociedade local. Classifique o texto criando uma lista temática concisa e coloque sua resposta à esta pergunta na chave "temas".
      2) O texto em análise apresenta um ou mais autores que defendem a importância de uma ou mais ações para a sociedade. Identifique esses autores de forma objetiva, ou seja, nome e cargo. Coloque sua resposta à esta pergunta na chave "autores" no formato de lista, mesmo que seja identificado apenas um autor.
      3) Se o texto em análise apresentar um ou mais destinatários das ações defendidas pelos autores. Identifique esses destinatários com seu nome, cargo e instituição a qual pertence. Explique a importância das ações defendidas pelos autores para esses destinatários no contexto de seus cargos e instituições. Utilize em sua resposta as chaves: "nome", "cargo", "instituicao", "importancia". Coloque sua resposta à esta pergunta na chave "destinatarios" no formato de lista, mesmo que seja identificado apenas um destinatário.
      4) Com base nas diversas áreas da gestão pública municipal, formule uma lista de possíveis instituições destinatárias não citadas pelos autores e justifique. Use as chaves "instituicao" e "justificativa". Coloque sua resposta à esta pergunta na chave "possiveis_destinatarios" no formato de lista.
      5) Caso o texto em análise apresente localidades, identifique essas localidades, sejam elas logradouros públicos, bairros ou mesmo prédios públicos e/ou privados, utilizando para isso a chave "nome", e adicionando o tipo identificado, utilizando a chave "tipo". Coloque sua resposta à esta pergunta na chave "localidades" no formato de lista.
      6) Formule a defesa do texto analisado com no máximo 150 palavras sobre a importância das ações defendidas pelos autores para os destinatários identificados. É muito importante que escreva essa defesa com um texto leve de forma que uma pessoa leiga no assunto entenda. Coloque sua resposta à esta pergunta na chave "analise" no formato de markdown.

      <CONTEXTO>{context}</CONTEXTO>"""

yaml_data = yaml.load(yaml_string, Loader=yaml.FullLoader)

In [24]:
yaml_data

{'PROMPTs': [{'id': 2,
   'status': True,
   'temperature': 0.1,
   'top_p': 0.95,
   'top_k': 40,
   'text': 'Você é um especialista em Gestão Pública Municipal com mais de 10 anos de conhecimento teórico e atividade prática.\nAnalise o conteúdo de <CONTEXTO></CONTEXTO> e escreva um JSON com as seguintes chaves: "temas", "autores", "destinatarios", "possiveis_destinatarios", "localidades" e "analise".\nPara preencher as chaves, siga as instruções abaixo em que, respectivamente, se referem às chaves acima citadas:\n1) Dentro das diversas áreas da gestão pública municipal, o texto em análise aborda a importância e faz solicitações à seus destinatários para a sociedade local. Classifique o texto criando uma lista temática concisa e coloque sua resposta à esta pergunta na chave "temas".\n2) O texto em análise apresenta um ou mais autores que defendem a importância de uma ou mais ações para a sociedade. Identifique esses autores de forma objetiva, ou seja, nome e cargo. Coloque sua respost

##### Extração de Temas gerado pelo código acima implementado já no PortalCMJ

In [49]:
from sapl.base.models import Metadata
temas_global = set()
mds = Metadata.objects.all()

for md in mds:
    temas = md.metadata.get('genia', {}).get('temas', [])
    for tema in temas:
        if tema not in temas_global:
            temas_global.add(' '.join(map(lambda x: x.capitalize() if len(x) > 3 else x, tema.split())))

temas_global = {
    k: k for k in sorted(temas_global)
    }

In [50]:
temas_global = list(temas_global.items())

In [51]:
temas_global[0]

('Abastecimento de Água', 'Abastecimento de Água')

In [56]:
md = Metadata.objects.filter(metadata__genia__temas__icontains=temas_global[0][0]).first()

In [12]:
for md in Metadata.objects.all():
    temas = md.metadata.get('genia', {}).get('temas', [])
    temas = set(temas)
    md.metadata['genia']['temas'] = list(temas)
    md.save()

In [2]:
from sapl.materia.models import AssuntoMateria
for a in AssuntoMateria.objects.filter(id__gt=4).order_by('assunto'):
    print(f'<ITEM>{a.assunto}</ITEM>')

<ITEM>Assistência Social e Inclusão</ITEM>
<ITEM>Atendimento ao Cidadão</ITEM>
<ITEM>Atendimento em Turno Integral</ITEM>
<ITEM>Bem-Estar Animal</ITEM>
<ITEM>Conselhos Municipais</ITEM>
<ITEM>Convênios e Parcerias</ITEM>
<ITEM>Cultura e Patrimônio</ITEM>
<ITEM>Democratização do Acesso a Cargos Públicos</ITEM>
<ITEM>Desenvolvimento Econômico e Social</ITEM>
<ITEM>Economia</ITEM>
<ITEM>Economia Solidária</ITEM>
<ITEM>Educação</ITEM>
<ITEM>Espaços Públicos</ITEM>
<ITEM>Esporte e Lazer</ITEM>
<ITEM>Finanças e Orçamento</ITEM>
<ITEM>Fiscalização de Posturas</ITEM>
<ITEM>Gerenciamento de Riscos</ITEM>
<ITEM>Gestão de Contratos e Licitações</ITEM>
<ITEM>Gestão de Pessoas e Servidores</ITEM>
<ITEM>Gestão de Recursos Públicos</ITEM>
<ITEM>Gestão de Riscos</ITEM>
<ITEM>Governança e Gestão Pública</ITEM>
<ITEM>Infraestrutura Urbana e Mobilidade</ITEM>
<ITEM>Legislação e Normas</ITEM>
<ITEM>Meio Ambiente e Sustentabilidade</ITEM>
<ITEM>Participação e Controle Social</ITEM>
<ITEM>Planejamento Urban

In [19]:
outra_classificacao = {
  "classificacao_documentos": [
    {
      "grupo": "Infraestrutura e Serviços Urbanos",
      "itens": [
        { "subgrupo": "Manutenção Viária", "descricao": "(Recapeamento, tapa-buracos, sinalização)" },
        { "subgrupo": "Iluminação Pública", "descricao": "(Reparos, expansão, modernização)" },
        { "subgrupo": "Limpeza Urbana", "descricao": "(Coleta de lixo, varrição, descarte irregular)" },
        { "subgrupo": "Drenagem Urbana", "descricao": "(Bueiros, galerias, enchentes)" },
        { "subgrupo": "Pavimentação", "descricao": "(Novas ruas, calçadas, acessibilidade)" },
        { "subgrupo": "Abastecimento de Água", "descricao": "(Reparos, vazamentos, novas ligações)" },
        { "subgrupo": "Esgotamento Sanitário", "descricao": "(Rede coletora, tratamento, fossas)" },
        { "subgrupo": "Transporte Público", "descricao": "(Linhas, horários, pontos de ônibus)" },
        { "subgrupo": "Mobilidade Urbana", "descricao": "(Ciclovias, trânsito, estacionamento)" },
        { "subgrupo": "Fiscalização de Obras", "descricao": "(Licenças, irregularidades, segurança)" },
        { "subgrupo": "Parques e Jardins", "descricao": "(Manutenção, criação, revitalização)" },
        { "subgrupo": "Cemitérios", "descricao": "(Manutenção, ampliação, concessões)" }
      ]
    },
    {
      "grupo": "Saúde",
      "itens": [
        { "subgrupo": "Atendimento Primário", "descricao": "(Postos de saúde, consultas, vacinação)" },
        { "subgrupo": "Especialidades Médicas", "descricao": "(Consultas, exames, encaminhamentos)" },
        { "subgrupo": "Saúde Mental", "descricao": "(Atendimento, CAPS, dependência química)" },
        { "subgrupo": "Vigilância Sanitária", "descricao": "(Fiscalização, alvarás, licenças)" },
        { "subgrupo": "Vigilância Epidemiológica", "descricao": "(Doenças, endemias, campanhas)" },
        { "subgrupo": "Farmácia Básica", "descricao": "(Disponibilidade de medicamentos, distribuição)" },
        { "subgrupo": "Urgência e Emergência", "descricao": "(Pronto Atendimento, SAMU, hospitais)" }
      ]
    },
    {
      "grupo": "Educação",
      "itens": [
        { "subgrupo": "Ensino Fundamental", "descricao": "(Matrículas, estrutura, merenda)" },
        { "subgrupo": "Educação Infantil", "descricao": "(Creches, pré-escola, vagas)" },
        { "subgrupo": "Ensino Médio", "descricao": "(Convênios, transporte escolar)" },
        { "subgrupo": "Educação de Jovens e Adultos", "descricao": "(EJA, cursos profissionalizantes)" },
        { "subgrupo": "Infraestrutura Escolar", "descricao": "(Reformas, equipamentos, acessibilidade)" },
        { "subgrupo": "Transporte Escolar", "descricao": "(Rotas, segurança, veículos)" },
        { "subgrupo": "Qualidade do Ensino", "descricao": "(Programas, formação de professores)" }
      ]
    },
    {
        "grupo": "Assistência Social",
        "itens": [
            { "subgrupo": "Benefícios Sociais", "descricao":"(Bolsa Família, Auxílio Brasil, outros)"},
            { "subgrupo": "CRAS e CREAS", "descricao":"(Atendimento, programas, encaminhamentos)"},
            { "subgrupo": "População em Situação de Rua", "descricao":"(Acolhimento, políticas, reinserção)"},
            { "subgrupo": "Conselho Tutelar", "descricao": "(Denúncias, acompanhamento, proteção)"},
            { "subgrupo": "Inclusão Social", "descricao": "(Pessoas com deficiência, minorias)"}
        ]
    },
   {
        "grupo": "Planejamento e Finanças",
        "itens": [
            { "subgrupo": "Orçamento Municipal", "descricao": "(Execução, remanejamento, investimentos)" },
            { "subgrupo": "Planejamento Urbano", "descricao": "(Plano Diretor, uso e ocupação do solo)" },
            { "subgrupo": "Licitações e Contratos", "descricao": "(Processos, irregularidades, fiscalização)" },
            { "subgrupo": "Tributação", "descricao": "(IPTU, ISS, taxas)" },
            { "subgrupo": "Contabilidade Pública", "descricao": "(Prestação de contas, auditoria)"}
        ]
    },
    {
      "grupo": "Outros",
      "itens": [
        { "subgrupo": "Segurança Pública", "descricao": "(Guarda Municipal, iluminação, videomonitoramento)" },
        { "subgrupo": "Meio Ambiente", "descricao": "(Licenciamento, fiscalização, preservação)" },
        { "subgrupo": "Cultura e Lazer", "descricao": "(Eventos, espaços culturais, esporte)" },
        { "subgrupo": "Desenvolvimento Econômico", "descricao": "(Incentivos, empreendedorismo, emprego)" }
      ]
    }
  ]
}

In [ ]:
from sapl.materia.models import AssuntoMateria
for a in AssuntoMateria.objects.filter(id__gt=4).order_by('assunto'):
    print(f'<ITEM>{a.assunto}</ITEM>')


In [8]:
from sapl.materia.models import AssuntoMateria, MateriaAssunto
from sapl.rules.apps import reset_id_model

AssuntoMateria.objects.filter(id__gt=4).delete()
reset_id_model(AssuntoMateria)

for a in AssuntoMateria.objects.filter(id__gt=4).order_by('assunto'):
    print(f'<ITEM>{a.assunto}</ITEM>')



(240, {'materia.MateriaAssunto': 202, 'materia.AssuntoMateria': 38})

In [5]:
AssuntoMateria.objects.values_list('id', flat=True)

<QuerySet [3, 1, 4, 2]>

In [10]:
MateriaAssunto.objects.all().count()

59

In [11]:
from sapl.materia.models import Tramitacao

In [14]:
Tramitacao.objects.filter(unidade_tramitacao_destino_id=13)

<QuerySet [<Tramitacao: Projeto de Lei Ordinária do Legislativo nº 21 de 2023 | Proposição retirada pelo autor | 03/10/2023>, <Tramitacao: Projeto de Resolução nº 7 de 2023 | Proposição apresentada em Plenário | 09/05/2023>, <Tramitacao: Projeto de Resolução nº 9 de 2021 | Aguardando emissão de parecer das comissões | 05/10/2021>, <Tramitacao: Projeto de Resolução nº 6 de 2021 | Proposição retirada pelo autor | 27/09/2021>, <Tramitacao: Projeto de Resolução nº 8 de 2021 | Proposição retirada pelo autor | 15/09/2021>, <Tramitacao: Projeto de Lei Ordinária do Executivo nº 35 de 2019 | Aguardando a inclusão na ordem do dia | 25/06/2019>, <Tramitacao: Projeto de Resolução nº 6 de 2018 | Prazo Regimental | 05/06/2018>, <Tramitacao: Projeto de Resolução nº 8 de 2014 | Proposição retirada pelo autor | 29/08/2014>]>